In [7]:
# scraper.py
#
# Scrapes data about guitars and reviews from a few different sources,
# then uses that data to populate an EdgeDB instance
#
# Much of the scraper code is just refactoring of Joe's GuitarCenter code, 
# put into a little more organized piece of code


from selenium import webdriver
from selenium.webdriver.firefox.options import Options
# import time
from bs4 import BeautifulSoup
import edgedb
import os

import numpy as np
# import json


In [2]:

# local import
import scrape_utils
# import class_definitions


In [3]:


# create new instance of firefox driver -- this should be the geckodriver
options = Options()
options.binary_location = r"C:\\Program Files\\Mozilla Firefox\\firefox.exe"
options.headless = True
driver = webdriver.Firefox(executable_path="C:\Program Files\GeckoDriver\geckodriver.exe", options=options)


In [4]:

# ---------------------------------------------------
# start with guitarcenter
# ---------------------------------------------------

url_list = [] #  list of guitar urls
guitars = [] # list of guitars
# iterate over the range of "Nao" values, get links to all guitars
# for ii in range(0, 4400, 100):
for ii in range(0,100,100): # temp testing
    html = scrape_utils.gc_get_browsing_pages(driver, ii) # get the html doc

    url_list.append(scrape_utils.gc_extract_links(html)) # append the list of matches

# make it unique -- without numpy
url_list = list(set([item for sublist in url_list for item in sublist]))

In [3]:
client = edgedb.create_client(dsn='MSDS_459')

In [ ]:
client.query(""" INSERT ReviewSource {
                    name := <str>'Guitar Center',
                    sourceType := <default::SourceType>'Vendor',
            } UNLESS CONFLICT """)

# create a "Guitar Center" vendor
client.query(""" INSERT Vendor {
                    name := <str>'Guitar Center',
            } UNLESS CONFLICT """)

In [ ]:
from importlib import reload
reload(scrape_utils)
reload(scrape_utils.class_definitions)
# reload(class_definitions)

In [ ]:
for url_i,url in enumerate(url_list):
    if url_i < 100:
        html = scrape_utils.gc_get_all_reviews(driver, url)  
        reviews = scrape_utils.gc_extract_review_info(html) # parse the review info
        guitar = scrape_utils.gc_extract_guitar_info(url, html) # parse the specs for the guitar

        guitar_id = guitar.insert(client)


        for review in reviews:
            review.insert(guitar_id, client)

In [6]:
url = url_list[1]
html = scrape_utils.gc_get_all_reviews(driver, url)  
reviews = scrape_utils.gc_extract_review_info(html) # parse the review info
guitar = scrape_utils.gc_extract_guitar_info(url, html) # parse the specs for the guitar

In [7]:
guitar_id = guitar.insert(client)

In [8]:
for review in reviews:
    review.insert(guitar_id, client)

In [ ]:
rating = review.rating if review.rating is not None else float()
rev_date = review.date if review.date is not None else str()
pros = review.pros if len(review.pros) > 0 else edgedb.Set()
cons = review.cons if len(review.cons) > 0 else edgedb.Set()
best_for = review.best_for if len(review.best_for) > 0 else edgedb.Set()
text = review.text if review.text is not None else str()

query_str = """INSERT Review {
                normalized_rating := <float64>$rating,
                date := <std::datetime>$rev_date, 
                pros := <array<str>>$pros,
                cons := <array<str>>$cons,
                best_for := <array<str>>$best_for,
                written_review := <str>$text,
                guitar := (
                    SELECT Guitar
                    filter .id = <uuid>$guitar_id
                    ),
                source :=(
                    SELECT ReviewSource
                    filter .name = <str>$review_source
                ),
                }
                """
return_val = client.query(query_str, rating=rating, rev_date=rev_date, pros=pros,\
             cons = cons, best_for = best_for, text = text, guitar_id = guitar_id[0].id, review_source = review.review_source)


In [ ]:
review.review_source

In [ ]:
query_str = """INSERT Review {
                normalized_rating := <float64>$rating,
                date := <std::datetime>$rev_date, 
                pros := <array<str>>$pros,
                cons := <array<str>>$cons,
                best_for := <array<str>>$best_for,
                written_review := <str>$text,
                guitar := (
                    SELECT Guitar
                    filter .id = <uuid>$guitar_id
                    ),
                source :=(
                    SELECT ReviewSource
                    filter .name = <str>$review_source
                ),
                }
                """
return_val = client.query(query_str, rating=rating, rev_date=rev_date, pros=pros,\
             cons = cons, best_for = best_for, text = text, guitar_id = guitar_id, review_source = review.review_source)


In [31]:
import re

In [ ]:
match = re.search(r'[C|c]lassical',guitar.model)

In [ ]:
match.group(0)

In [9]:

driver.close()
client.close()

In [13]:
client = edgedb.create_client(dsn='MSDS_459')

query_string = '''WITH pickups := 
            (SELECT Guitar.pickups filter Guitar.pickups != '')
            
            SELECT DISTINCT pickups;
            '''
cutaways = client.query(query_string)

In [8]:
resp = client.query(query_string)

In [14]:
cutaways

[]

In [5]:
search_dict = dict()

guitar_props = ['body_shape','cutaway','pickups']

for prop in guitar_props:
    query_string = f"""WITH property :=
            (SELECT Guitar.{prop} filter Guitar.{prop} != '')
            SELECT DISTINCT property"""

    search_dict[prop] = client.query(query_string)    


In [62]:
# find valid entries for all "string" fields
str_props = ['body_shape','cutaway','pickups', 'type', 'country_of_origin']
for prop in str_props:
    query_string = f'''WITH prop :=
            (SELECT str_lower(Guitar.{prop}) filter Guitar.{prop} != '')
            SELECT DISTINCT prop'''
    search_dict[prop] = client.query(query_string)

In [6]:
search_dict

{'body_shape': ['000-14 Fret',
  'Concert',
  'Dreadnought, 14th Fret',
  'Other',
  'Semi-hollow or chambered body',
  'Solid body'],
 'cutaway': ['Non-cutaway', 'Single cutaway', 'Single-cutaway'],
 'pickups': []}

In [4]:
num_props = ['num_strings','scale_length','num_frets']
for prop in num_props:
    query_string = f'''WITH prop :=
            (SELECT Guitar.{prop} filter Guitar.{prop} != 0)
            SELECT DISTINCT prop'''
    search_dict[prop] = client.query(query_string)

search_dict

NameError: name 'search_dict' is not defined

In [39]:
test_str = 'Acoustic-electric'
match = re.search('[A|a]coustic[ |-][E|e]lectric', test_str).group(0)
match

'Acoustic-electric'

In [40]:
search_dict

{'body_shape': ['000-14 Fret',
  'Concert',
  'Dreadnought, 14th Fret',
  'Other',
  'Semi-hollow or chambered body',
  'Solid body'],
 'cutaway': ['Non-cutaway', 'Single cutaway', 'Single-cutaway'],
 'pickups': [],
 'type': ['Acoustic', 'Acoustic Electric', 'Classical', 'Electric', 'unknown'],
 'country_of_origin': [],
 'num_strings': [],
 'scale_length': [],
 'num_frets': []}

In [10]:
query_str = 'SELECT Guitar {model, description} filter .type in {"Electric"} limit 5;'
with edgedb.create_client(dsn='MSDS_459') as client:

    guitars = client.query(query_str)

In [13]:
type(guitars[0])

edgedb.Object

In [18]:
dir(guitars[0])

['description', 'id', 'model']

In [20]:
guitar_dict = {}
for i_guitar,guitar in enumerate(guitars):
    guitar_dict[i_guitar] = {'model':guitar.model, 'description':guitar.description}

In [21]:
guitar_dict

{0: {'model': 'SP7F Spectre Electric Guitar', 'description': ''},
 1: {'model': 'G&amp;L Fullerton Deluxe ASAT Special Left Handed Electric Guitar',
  'description': 'The left-handed Gu0026L ASAT Special just might be the quintessential Gu0026L guitar, viewed by many as the ultimate single-cutaway. Back in 1985, this model was launched under the Gu0026L Broadcaster name, which prompted a response from Fred Gretsch of Gretsch Guitars over the name as Broadkaster is owned by Gretsch. The name then became “ASAT” which is US military shorthand for an “Anti-Satellite” missile. The ASAT Special, as the model became known, features jumbo single-coil Magnetic Field Design pickups, each with a narrow bobbin aperture for a coil which is shallow yet wide. In this regard, an MFD jumbo single coil is similar to a P-90 or Jazzmaster pickup, but the MFD principle provides a punchier bottom end and brighter top end that sparkles with harmonics. With the addition of his innovative Saddle-Lock bridge wh

In [4]:
with open("product_urls.txt", "r") as file:
    url_list = file.read().splitlines()

In [6]:
n_urls = len(url_list)
for i in range(n_urls-10, n_urls): # can modify this to only download a subset of the urls at a time
    # Construct the full URL
    url_partial = url_list[i]
    print(f"{i}: {url_partial}")
    url = "https://www.guitarcenter.com" + url_partial
    driver.get(url)
    html = scrape_utils.gc_get_all_reviews(driver, url)

    # Save the HTML to a file.
    # Everything before the slash is the directory, everything after except the ".gc" is the file name.
    # If the subdirectory doesn't exist, create it.
    subDir = url_partial[1:url_partial.rfind('/')]
    if not os.path.exists(f"{saveDir}/{subDir}"):
        os.makedirs(f"{saveDir}/{subDir}")
    with open(f"{saveDir}/{url_partial[1:-3]}.html", "w", encoding='utf-8') as file:
        file.write(html)

5582: /PRS/S2-Singlecut-McCarty-594-Electric-Guitar-McCarty-Sunburst-1500000315350.gc


NameError: name 'os' is not defined

In [8]:
driver.close()